In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
import pandas as pd


In [55]:
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_no_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\combine_all_class_with_label_apply_fft.csv'
# file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_combine_all_class_with_label_applied_fft.csv'
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win5_lab50.csv'
df = pd.read_csv(file_path)

In [56]:
# -- Prepare for modeling --
# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
#    - We'll drop 'start_time' because it's just a reference
#    - We'll keep 'label_encoded' as the target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,   # 20% for testing
    random_state=42  # for reproducibility
)

# Apply min-max normalization (0-1) to these columns
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# 4) Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5) Evaluate
y_pred = model.predict(scaler.transform(X_test))
print(classification_report(y_test, y_pred, target_names=le.classes_))

                             precision    recall  f1-score   support

            climbing_stairs       0.84      0.72      0.78        53
          descending_stairs       0.90      0.88      0.89        73
                    nothing       1.00      0.93      0.96        54
                    running       0.98      1.00      0.99        53
sitting_standing_transition       0.93      0.98      0.95        64
                    walking       0.86      0.97      0.91        67

                   accuracy                           0.91       364
                  macro avg       0.92      0.91      0.91       364
               weighted avg       0.92      0.91      0.91       364



In [ ]:
!pip install tensorflow onnx onnxmltools skl2onnx nyoka scikit-learn joblib pandas numpy onnxruntime jpmml-evaluator

^C


In [1]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
import onnx
import onnxmltools
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from nyoka import skl_to_pmml

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# -- Load Data --
file_path = r'D:\git_repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win5_lab50.csv'
df = pd.read_csv(file_path)

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5) Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 6) Save the trained model
joblib.dump(model, "random_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

# 7) Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


                             precision    recall  f1-score   support

            climbing_stairs       0.84      0.72      0.78        53
          descending_stairs       0.90      0.88      0.89        73
                    nothing       1.00      0.93      0.96        54
                    running       0.98      1.00      0.99        53
sitting_standing_transition       0.93      0.98      0.95        64
                    walking       0.86      0.97      0.91        67

                   accuracy                           0.91       364
                  macro avg       0.92      0.91      0.91       364
               weighted avg       0.92      0.91      0.91       364



In [53]:
# IF USE TORCH
# import torch
# import torch.nn as nn
# import torch.optim as optim

# class RF_NN(nn.Module):
#     def __init__(self, input_size, num_classes):
#         super(RF_NN, self).__init__()
#         self.fc1 = nn.Linear(input_size, 160)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(160, num_classes)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.softmax(x)
#         return x

# # Create the PyTorch model
# input_size = X_train.shape[1]
# num_classes = len(set(y))
# torch_model = RF_NN(input_size, num_classes)


In [5]:
import numpy as np
import tensorflow as tf
import random

# 1️⃣ Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 2️⃣ Define model with consistent weight initialization
def rf_to_nn():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(30, activation="relu", input_shape=(X_train.shape[1],), kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(22, activation="relu", kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        # tf.keras.layers.Dense(10, activation="relu", kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        tf.keras.layers.Dense(len(set(y)), activation="softmax", kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED))
    ])

# 3️⃣ Create and compile model
tf_model = rf_to_nn()
tf_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
tf_model.summary()

# 4️⃣ Train model
tf_model.fit(X_train, model.predict_proba(X_train), epochs=50, verbose=1)

# 5️⃣ Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()
tflite_model_path = "random_forest.tflite"

# 6️⃣ Save TFLite model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved: {tflite_model_path}")


d:\anaconda\envs\final_project_dsp\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 30)             │         3,810 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 22)             │           682 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 6)              │           138 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,630 (18.09 KB)

 Trainable params: 4,630 (18.09 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2179 - loss: 1.7340
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4517 - loss: 1.4883
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6621 - loss: 1.3093
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7638 - loss: 1.1043
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8017 - loss: 0.9278
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8277 - loss: 0.8017
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8553 - loss: 0.7237
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8767 - loss: 0.6730
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8930 - loss: 0.6379
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9031 - loss: 0.6121
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9127 - loss: 0.5928
Epoch 12/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9164 - lo

INFO:tensorflow:Assets written to: C:\Users\tanar\AppData\Local\Temp\tmpntsp4rce\assets


Saved artifact at 'C:\Users\tanar\AppData\Local\Temp\tmpntsp4rce'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 126), dtype=tf.float32, name='keras_tensor_12')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  2743938067168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2743938056960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2743938068400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2743938521696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2743938521168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2743938518176: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved: random_forest.tflite


In [ ]:
# ✅ **Test TFLite Model**
import tensorflow.lite as tflite

tflite_model_path = "random_forest.tflite"
interpreter = tflite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

tflite_preds = []
for sample in X_test.astype(np.float32):
    sample = np.expand_dims(sample, axis=0)
    interpreter.set_tensor(input_index, sample)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    tflite_preds.append(np.argmax(pred))

print("TFLite Model Performance:\n", classification_report(y_test, tflite_preds, target_names=le.classes_))



TFLite Model Performance:
                              precision    recall  f1-score   support

            climbing_stairs       0.86      0.92      0.89        53
          descending_stairs       0.97      0.86      0.91        73
                    nothing       1.00      0.94      0.97        54
                    running       0.98      1.00      0.99        53
sitting_standing_transition       0.94      0.98      0.96        64
                    walking       0.93      0.97      0.95        67

                   accuracy                           0.95       364
                  macro avg       0.95      0.95      0.95       364
               weighted avg       0.95      0.95      0.94       364

